In [1]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/*.pkl'))
len(files)

23600

In [2]:
from datetime import datetime, timedelta
import pathlib

atleast = datetime(2023, 8, 5)

filtered = []
for f in files:
    fname = pathlib.Path(f)
    if datetime.fromtimestamp(fname.stat().st_mtime) >= atleast:
        filtered.append(f)
        
files = sorted(filtered)
len(files)

6976

In [3]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [4]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [6]:
# lens = 0
# for f in files:
#     with open(f, 'rb') as fopen:
#         data = pickle.load(fopen)
#     lens += len(data)
    
# lens

In [7]:
with open(files[4], 'rb') as fopen:
    data = pickle.load(fopen)
    
len(data)

5

In [8]:
data[0]

{'wav_data': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/Beza_Warna_Uniform_Pada_Pengakap_[Tuan_Hilmi]_Apa_Maksud_Disebaliknya?_[FmhdLuLqz-U]/0.mp3',
 'start': 0.0,
 'asr_model': ([[('<|ms|>', 1.0),
    ('<|id|>', 0.00016880035400390625),
    ('<|ta|>', 9.012222290039062e-05),
    ('<|en|>', 6.580352783203125e-05),
    ('<|nn|>', 4.839897155761719e-05),
    ('<|ko|>', 4.267692565917969e-05),
    ('<|vi|>', 3.528594970703125e-05),
    ('<|th|>', 3.314018249511719e-05),
    ('<|fr|>', 2.753734588623047e-05),
    ('<|ar|>', 2.753734588623047e-05),
    ('<|tl|>', 2.586841583251953e-05),
    ('<|jw|>', 2.4318695068359375e-05),
    ('<|zh|>', 1.7762184143066406e-05),
    ('<|cy|>', 1.7762184143066406e-05),
    ('<|ja|>', 1.6689300537109375e-05),
    ('<|km|>', 1.2993812561035156e-05),
    ('<|tr|>', 1.2993812561035156e-05),
    ('<|ro|>', 1.1444091796875e-05),
    ('<|ru|>', 8.940696716308594e-06),
    ('<|pt|>', 5.781650543212891e-06),
    ('<|hi|>', 5.424022674560547e-06),
  

In [8]:
import IPython.display as ipd
ipd.Audio(data[0]['wav_data'])

In [9]:
data[0]['asr_model'][1][1][0]

-0.1875

In [10]:
data[0]['asr_model'][2][1][0]

-0.7401315569877625

In [11]:
data[0]['asr_model'][1][0]

' Pernah tak kawan-kawan semua ternampak ahli-ahli pengakab yang memakai uniform berwarna kelabu atau berwarna putih atau berwarna biru muda Jadi dalam video kali ini kita akan berkongsi apakah perbezaan antara ketiga-tiga uniform ini dan perbezaan dari segi aktiviti'

In [12]:
ms_score = data[-3]['asr_model'][1][1][0]
en_score = data[-3]['asr_model'][2][1][0]

if ms_score >= en_score:
    selected = data[-3]['asr_model'][1][0]
else:
    selected = data[-3]['asr_model'][2][0]
    
selected

' keselamatan di air. Jadi untuk pengakab udara pula mereka mempunyai fokus-fokus berkaitan dengan anatomi pesawat, ilmu meteorologi, teknikal pesawat dan sebagainya. Jadi di sini ada variasi antara aktiviti-aktiviti yang dijalankan di mana bergantung kepada minat seseorang itu mereka boleh menyertai mana-mana kumpulan tersebut sama ada pengakab, pengakab udara atau pengakab laut. Untuk yang baru nak menyertai pengakab ini saya cadangkan kita perlu lihat'

In [7]:
f = data[-3]['wav_data']
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio
ipd.Audio(y, rate = sr)

In [10]:
import soundfile as sf

sf.read(f)

(array([ 0.00000000e+00, -1.03457132e-10, -2.55489907e-10, ...,
         3.05413721e-07, -8.61785111e-07, -6.71096416e-07]),
 22050)

In [36]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

2023-08-08 14:00:47.762447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 14:00:47.960247: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-08 14:00:48.972728: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-08 14:00:48.973080: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [37]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [32]:
directory = '/home/husein/ssd3/stt/whisper-v6'

In [33]:
# !rm -rf {directory}
!mkdir {directory}

In [34]:
import torchaudio
import torch
import os

In [31]:
lang_prob = {}
for l in data[0]['asr_model'][0][0]:
    lang_prob[l[0]] = l[1]
    
max(lang_prob, key=lang_prob.get)

'<|ms|>'

In [41]:
dataset = []
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang_prob = {}
        for l in data[i]['asr_model'][0][0]:
            lang_prob[l[0]] = l[1]

        lang = max(lang_prob, key=lang_prob.get)
        if lang not in {'<|ms|>', '<|en|>'}:
            continue
            
        ms_score = data[i]['asr_model'][1][1][0]
        en_score = data[i]['asr_model'][2][1][0]

        if ms_score >= en_score:
            selected = data[i]['asr_model'][1][0]
        else:
            selected = data[i]['asr_model'][2][0]
            
        t = selected
        cleaned_t = preprocessing_text(t)
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data']))['array']
        dataset.append(
            {
                'file': data[i]['wav_data'],
                'text': t,
                'cleaned': cleaned_t,
                'no': no,
                'i': i,
                'len': len(audio) / 16000
            }
        )
        
    break

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 43.75it/s]


In [42]:
dataset

[{'file': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/Beza_Simpan_dan_Labur_EMAS_[Amanina_Matali]_Anda_Yang_Mana_Satu?_[hWjj2Fd7iY0]/0.mp3',
  'text': ' Saya selalu dapat soalan macam ni, Cik Nina boleh ajar saya buat pelaburan emas? Saya akan tanya balik, dah mula simpan emas ke belum? Percaya atau tidak, majlis itu akan jawab mereka tak pernah simpan emas lagi.',
  'cleaned': 'saya selalu dapat soalan macam ni cik nina boleh ajar saya buat pelaburan emas saya akan tanya balik dah mula simpan emas ke belum percaya atau tidak majlis itu akan jawab mereka tak pernah simpan emas lagi',
  'no': 0,
  'i': 0,
  'len': 11.807375},
 {'file': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/Beza_Simpan_dan_Labur_EMAS_[Amanina_Matali]_Anda_Yang_Mana_Satu?_[hWjj2Fd7iY0]/1.mp3',
  'text': " Newbie yang baru nak kenal dunia emas sebabnya akan terliru 2 terma ni. Sakalkan sama tapi tak serupa ye? Nak tahu bezanya? Let's joke! Kita mulakan dengan penyimpan emas dulu ye. Penyim

In [ ]:
len(dataset)

In [ ]:
import json

with open('dataset-whisper-stt-2023-06-19.json', 'w') as fopen:
    json.dump(dataset, fopen)

In [ ]:
!ls -lh dataset-whisper-stt-2023-06-19.json